In [2]:
client_id = ""
client_secret = ""
tenant_id = ""

# 1. Acquire a access token

In [3]:
import msal
import requests

# Create a confidential client application
app = msal.ConfidentialClientApplication(
    client_id,
    authority=f"https://login.microsoftonline.com/{tenant_id}",
    client_credential=client_secret
)

# Acquire a token
result = app.acquire_token_for_client(scopes=["https://graph.microsoft.com/.default"])

if "access_token" in result:
    access_token = result['access_token']
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
else:
    print("Error acquiring token:", result.get("error"), result.get("error_description"))


ValueError: Your given address (https://login.microsoftonline.com/) should consist of an https url with a minimum of one segment in a path: e.g. https://login.microsoftonline.com/{tenant} or https://{tenant_name}.ciamlogin.com/{tenant} or https://{tenant_name}.b2clogin.com/{tenant_name}.onmicrosoft.com/policy

# 2. Graph API Call
## 2-1. Get Users and the last SignIns in Security Groups

In [9]:
import requests
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType
from pyspark.sql.functions import current_date, lower, expr, col

StatementMeta(, 88d3d111-ce16-4f30-998b-8af2f3ad7c3f, 11, Finished, Available, Finished)

In [5]:
# Load security_groups table from Lakehouse
df_groups = spark.read.table("security_groups")
groups_list = df_groups.select("group_id", "group_name").collect()

# Define schema for group users
group_users_schema = StructType([
    StructField("group_id", StringType(), True),
    StructField("group_name", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("userPrincipalName", StringType(), True),
    StructField("displayName", StringType(), True),
    StructField("lastSignInDateTime", StringType(), True)
])

# Make sure to have access_token defined in advance
# access_token = "<Your_Access_Token>"
headers = {
    'Authorization': f'Bearer {access_token}'
}

all_users_data = []

# Loop through each group and fetch its members
for group in groups_list:
    group_id = group['group_id']
    group_name = group['group_name']
    
    print(f"Processing group: {group_name} ({group_id})")

    members_url = f"https://graph.microsoft.com/v1.0/groups/{group_id}/members?$select=id,displayName,userPrincipalName"
    while True:
        members_response = requests.get(members_url, headers=headers)
        members_response.raise_for_status()
        members_data = members_response.json()

        # For each member, fetch sign-in activity
        for member in members_data.get("value", []):
            if member.get("@odata.type") == "#microsoft.graph.user":
                user_id = member["id"]

                user_url = f"https://graph.microsoft.com/beta/users/{user_id}?$select=displayName,userPrincipalName,signInActivity"
                user_response = requests.get(user_url, headers=headers)
                user_response.raise_for_status()
                user_data = user_response.json()

                last_signin = user_data.get("signInActivity", {}).get("lastSignInDateTime", "No Data")

                all_users_data.append({
                    "group_id": group_id,
                    "group_name": group_name,
                    "user_id": user_id,
                    "userPrincipalName": user_data.get("userPrincipalName", ""),
                    "displayName": user_data.get("displayName", ""),
                    "lastSignInDateTime": last_signin
                })

        # Handle pagination
        next_link = members_data.get("@odata.nextLink")
        if next_link:
            members_url = next_link
        else:
            break

# Create DataFrame from collected user data
df_group_users = spark.createDataFrame(all_users_data, schema=group_users_schema)

# Add execution_date and domain_lc columns
df_group_users = df_group_users.withColumn("execution_date", current_date()) \
                                 .withColumn("domain_lc", lower(expr("split(userPrincipalName, '@')[1]")))

# Save result as a Lakehouse table
df_group_users.write.mode("overwrite").saveAsTable("security_group_users")

display(df_group_users)


StatementMeta(, 88d3d111-ce16-4f30-998b-8af2f3ad7c3f, 7, Finished, Available, Finished)

Processing group: All Users (074782bf-4e04-4a57-af0e-97e0c6a37d56)
Processing group: PurviewGroup (636eb69b-0acd-4850-a7ec-08cbde71a672)


SynapseWidget(Synapse.DataFrame, 7631a2bc-b77d-4298-95b4-94dcb70a957f)

## 2-2. Get Subscribed Skus

In [10]:
# Define the schema
schema_subscribedSkus = StructType([
    StructField("accountName", StringType(), True),
    StructField("accountId", StringType(), True),
    StructField("appliesTo", StringType(), True),
    StructField("capabilityStatus", StringType(), True),
    StructField("consumedUnits", IntegerType(), True),
    StructField("id", StringType(), True),
    StructField("skuId", StringType(), True),
    StructField("skuPartNumber", StringType(), True),
    StructField("subscriptionIds", ArrayType(StringType()), True),
    StructField("prepaidUnits", StructType([
        StructField("enabled", IntegerType(), True),
        StructField("suspended", IntegerType(), True),
        StructField("warning", IntegerType(), True),
        StructField("lockedOut", IntegerType(), True)
    ])),
    StructField("servicePlans", ArrayType(StructType([
        StructField("servicePlanId", StringType(), True),
        StructField("servicePlanName", StringType(), True),
        StructField("provisioningStatus", StringType(), True),
        StructField("appliesTo", StringType(), True)
    ])), True)
])

# Make a request to the Microsoft Graph API
response = requests.get(f'https://graph.microsoft.com/v1.0/subscribedSkus', headers=headers)

if response.status_code == 200:
    #dataframes[f'df_{table_name}'] = response.json().get('value', [])
    #print(dataframes[f'df_{table_name}'])
    df_subscribedSkus = spark.createDataFrame(response.json().get('value', []), schema=schema_subscribedSkus)
else:
    print(f"Error: {response.text}")

df_subscribedSkus_parsed = df_subscribedSkus.select(
    "skuId",
    col("prepaidUnits.enabled").alias("enabled"),
    col("prepaidUnits.suspended").alias("suspended"),
    col("prepaidUnits.warning").alias("warning"),
    col("prepaidUnits.lockedOut").alias("lockedOut"),
).withColumn("execution_date", current_date())

#print(response.json().get('value', []))
display(df_subscribedSkus_parsed)


StatementMeta(, 88d3d111-ce16-4f30-998b-8af2f3ad7c3f, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5887f1e2-a3d0-4333-aed7-20d200780b71)

## 2-3. Get User License Assignments

In [11]:
# Convert the collected data (all_users_data) to a DataFrame with a defined schema
schema_assigned = StructType([
    StructField("skuId", StringType(), True),
    StructField("userPrincipalName", StringType(), True),
    StructField("displayName", StringType(), True),
    StructField("domain", StringType(), True),
])

def get_users_with_sku(access_token, sku_id):

    endpoint = (
        "https://graph.microsoft.com/v1.0/users"
        f"?$filter=assignedLicenses/any(a:a/skuId eq {sku_id})"
    )

    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    all_users = []
    while True:
        response = requests.get(endpoint, headers=headers)
        data = response.json()

        # Add the users from the current page
        all_users.extend(data.get("value", []))

        # Check if there's another page of data
        next_link = data.get("@odata.nextLink")
        if next_link:
            endpoint = next_link
        else:
            break

    return all_users

# Retrieve all SKU IDs from df_skus (convert them into a Python list)
sku_ids = [row["skuId"] for row in df_subscribedSkus.collect()]

# Temporary Python list to store data before creating the final DataFrame
all_users_data = []

for sku_id in sku_ids:
    users_with_sku = get_users_with_sku(access_token, sku_id)
    
    for user in users_with_sku:
        # Extract necessary information and store it in a list/dictionary format
        upn = user["userPrincipalName"]
        display_name = user.get("displayName", "")
        Domain = upn.split("@")[1]
        
        all_users_data.append({
            "skuId": sku_id,
            "userPrincipalName": upn,
            "displayName": display_name,
            "domain": Domain
        })

df_assigned_users = spark.createDataFrame(all_users_data, schema=schema_assigned)
df_assigned_users_date = df_assigned_users.withColumn("execution_date", current_date()) \
                                            .withColumn("domain_lc",lower(df_assigned_users["domain"]))

display(df_assigned_users_date)

StatementMeta(, 88d3d111-ce16-4f30-998b-8af2f3ad7c3f, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7c746695-d61f-4239-b66a-30fd19d09cb1)

# 3. Save Delta Table

In [12]:
df_group_users.write.mode("overwrite").format("delta").saveAsTable("graph_group_users")


StatementMeta(, 88d3d111-ce16-4f30-998b-8af2f3ad7c3f, 14, Finished, Available, Finished)

In [14]:
df_assigned_users_date.write.mode("overwrite").format("delta").saveAsTable("graph_assigned_users")

StatementMeta(, 88d3d111-ce16-4f30-998b-8af2f3ad7c3f, 16, Finished, Available, Finished)

In [15]:
df_subscribedSkus_parsed.write.mode("append").format("delta").saveAsTable("graph_subscribedSkus")

StatementMeta(, 88d3d111-ce16-4f30-998b-8af2f3ad7c3f, 17, Finished, Available, Finished)